## 01. Data I/O and Featurization

There are 12 Trp-Cage (TC5b) trajectories we will load and visualize.

If you haven't yet downloaded these files, please follow the instructions in `../TC5b-data`

In [20]:
import os, glob

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#import mdshare
import pyemma

# for visualization of molecular structures:
import nglview
import mdtraj
from threading import Timer
from nglview.player import TrajectoryPlayer

datadir = '../TC5b-data'
filenames = [os.path.join(datadir, s.strip()) for s in """p16959r9c29-whole.xtc 
p16959r9c83-whole.xtc 
p16959r9c100-whole.xtc
p16959r9c109-whole.xtc
p16959r9c127-whole.xtc
p16959r9c151-whole.xtc
p16959r9c166-whole.xtc
p16959r9c219-whole.xtc
p16959r9c312-whole.xtc
p16959r9c394-whole.xtc
p16959r9c631-whole.xtc
p16959r9c726-whole.xtc""".split('\n')]
print(filenames)

pdbfile = os.path.join(datadir, 'xtc_atoms.gro')

# Create a list of mdtraj Trajectory() objects
trajs = []
for fn in filenames:
    print(f'Loading {fn}... ', end='')
    trj = mdtraj.load(fn, top=pdbfile)
    
    print('Processing ...', end='')
    # 1. they have ions in them -- let's strip these out
    trj_protein = trj.atom_slice(trj.top.select("protein"))
    # 2. they are not centered - let's center them
    trj_protein.center_coordinates()
    # 3. they are not RMSD-aligned - let's center them on a frame at ~4 us
    trj_protein.superpose(trj_protein, frame=4000, parallel=True)
    trajs.append( trj_protein )
    print('...Done')


['../TC5b-data/p16959r9c29-whole.xtc', '../TC5b-data/p16959r9c83-whole.xtc', '../TC5b-data/p16959r9c100-whole.xtc', '../TC5b-data/p16959r9c109-whole.xtc', '../TC5b-data/p16959r9c127-whole.xtc', '../TC5b-data/p16959r9c151-whole.xtc', '../TC5b-data/p16959r9c166-whole.xtc', '../TC5b-data/p16959r9c219-whole.xtc', '../TC5b-data/p16959r9c312-whole.xtc', '../TC5b-data/p16959r9c394-whole.xtc', '../TC5b-data/p16959r9c631-whole.xtc', '../TC5b-data/p16959r9c726-whole.xtc']
Loading ../TC5b-data/p16959r9c29-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c83-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c100-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c109-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c127-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c151-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c166-whole.xtc... Processing ......Done
Loading ../TC5b-data/p16959r9c219-whole.xtc... Proce

In [21]:
widget = nglview.show_mdtraj(trajs[0])
p = TrajectoryPlayer(widget)
widget.add_ball_and_stick()
p.spin = True
# def stop_spin():
#     p.spin = False
#    widget.close()
# Timer(30, stop_spin).start()
widget

NGLWidget(max_frame=44183)

NGLWidget(max_frame=44183)

In [ ]:
#/usr/bin/python
'''
Tim Marshall
11/07/2021
tmchmbusiness@gmail.com
'''
import os, sys, glob, subprocess, time, csv, itertools, math
import numpy as np
import pyemma
import h5py

import mdtraj
'''This class is designed to intake simulation data from user specified dir and process it into featurized npy files
featurization is dependent on the function use.
'''

class organizer():
    def __init__(self):
        """Initial function.
        Parameters
        ----------

        """
        self.title='Featurizer to grab data from xtc files in user specified dir'
    def get_files(self, mydir, extensions):
        """Function to locate featurization relevant files from mydir. 
        
        All files are listed under user-defined mydir.
        
        Parameters
        ----------
        mydir: str
            This is path to the directories containing data in individual RUN directories
            |>mydir
                |>RUN0
                    |>*.xtc
                    |>*.gro
                    |>.ndx
                |>RUN1
                    |>*.xtc
        extensions: list(strs)
            A list of strings containing extensions to ['trajectory', 'structure' , 'index files'] as in ['.xtc','.gro',.'ndx']
            If you are working with P16958 gromacs style format, just leave this be. 
        """
        abs_path = os.path.abspath(f'{mydir}')    #translates user defined path into abssolute path. This prevents issues with user input for backslashes.
        dir_list = [f for f in os.listdir(abs_path) if os.path.isdir(os.path.join(abs_path, f))]
        
        out_list = [[[]  for _ in range(len(extensions))]  for _ in range(len(dir_list))]    #generated [[[ ]]] nested list for populating with extensions per RUN

        r=0    #iterator from 0 to len(dir_list), selects empty list index for file depositiona        

        ### Loops to itterate over all files in all directories inside user-defined {mydir}. 

        for ndir in dir_list:    
            ndir = f'{abs_path}/{ndir}'    #generates an absolute path to r^th RUN directory. Safety measure just in case. 
            for f in os.listdir(ndir):    #for all items(files and dirs) in r^th RUN dir
                if os.path.isfile(os.path.join(ndir, f)):    #for only the files in r^th RUN dir
                    file_path = [f'{ndir}/{f}']    #generate a complete absolute name to the file
                    for ext in range(len(extensions)):    #sorting into appropriate list based on user-defined file extensions.
                        if os.path.splitext(f)[1] == extensions[ext]:
                            out_list[r][ext].extend(file_path)    #save the file into the appropropriate list. 
                        else:
                            continue
            r+=1

        ### Vince Voelz's sorting for the traj files. orders things from 0..33 properly
        chignolin = True
        if chignolin == True:

            trajs_list = out_list[0]
            clone_indices = [ int(os.path.basename(filename).split('_C')[1].replace('.xtc','')) for filename in trajs_list ]
            sorted_traj_list = [None]*len(trajs_list)    
            for i in range(len(clone_indices)):
                sorted_traj_list[clone_indices[i]] = trajs_list[i]
            out_list[0] = sorted_traj_list
        else:
            pass
        ### end of dirty sorting. if working with a different dataset, remove in the future

        return dir_list, out_list

class featurizer(object):
    def __init__(self):
        """Initial function.
        Parameters
        ----------

        """
        self.title='Featurizer to grab data from xtc files in user specified dir'


    def featurizer(self, feat, out_list, out_name, save):
        """Calculates user-defined feature parameters.

        out_list: list(str)
            List of paths to trajectory files as indentified in the get_files function
        out_name: str
            Name of the feature to save npy file to. User preference
        """
        trajs_list = out_list[0]

        ### Vince Voelz's sorting for the traj files. orders things from 0..33 properly
        chignolin = True
        if chignolin == True:
            clone_indices = [ int(os.path.basename(filename).split('_C')[1].replace('.xtc','')) for filename in trajs_list ]
            sorted_traj_list = [None]*len(trajs_list)    
            for i in range(len(clone_indices)):
                sorted_traj_list[clone_indices[i]] = trajs_list[i]
            print(f'WARNING: You are parsing the CLN001 dataset. Vince Voelzs sorting algorithm applied\nHere is the list of trajectories being analyzed: {sorted_traj_list}')
        else:
            pass
        trajs_list = sorted_traj_list
        ### end of dirty sorting. if working with a different dataset, remove in the future
        
        feat_data = pyemma.coordinates.load(trajs_list, features=feat)
        
        if save == True:
            np.save(out_name,feat_data)
        return feat_data

    def score_cv(self, feat_data, dim, lag, number_of_splits, validation_fraction, out_name, save):
        """Compute a cross-validated VAMP2 score.

        We randomly split the list of independent trajectories into
        a training and a validation set, compute the VAMP2 score,
        and repeat this process several times.

        Parameters
        ----------
        data : list of numpy.ndarrays
            The input data.
        dim : int
            Number of processes to score; equivalent to the dimension
            after projecting the data with VAMP2.
        lag : int
            Lag time for the VAMP2 scoring.
        number_of_splits : int, optional, default=10
            How often do we repeat the splitting and score calculation.
        validation_fraction : int, optional, default=0.5
            Fraction of trajectories which should go into the validation
            set during a split.
        """
        nval = int(len(feat_data) * validation_fraction)
        scores = np.zeros(number_of_splits)
        for n, _ in enumerate(scores):
            ival = np.random.choice(len(feat_data), size=nval, replace=False)
            valid = [d for i, d in enumerate(feat_data) if i not in ival]
            vamp = pyemma.coordinates.vamp(valid, lag=lag, dim=dim, scaling = 'km')
            scores[n] = vamp.score([d for i, d in enumerate(feat_data) if i in ival])

        if save == True:
            np.save(out_name,scores)
        return scores

### TEMP FUNCTION STORAGE, NEED TO COMMENT ADN CLEAN BEFORE PUBLISHING

def tica(feat, lag_time, save, save_dir, save_name, feat_name):
    feat_list = np.ndarray.tolist(feat)
    tica = pyemma.coordinates.tica(feat_list, dim=8, lag=lag_time, stride=1, kinetic_map=True, commute_map=False)
    tica_getoutput = tica.get_output()
    if save == True:
        tica.save(f'{save_dir}/{feat_name}_tica_raw.h5', overwrite=True)
        np.save(f'{save_dir}/{feat_name}_tica_getoutput.npy', tica_getoutput)
    return tica_getoutput

def k_means(tica_getoutput, n_cluster, max_iter, save, save_dir, save_name, feat_name):
    cluster = pyemma.coordinates.cluster_kmeans(tica_getoutput, k=n_cluster, max_iter=max_iter)
    cluster_getoutput = cluster.get_output()
    cluster_dtrajs = cluster.dtrajs
    cluster_centers = cluster.clustercenters
    if save == True:
        cluster.save(f'{save_dir}/{feat_name}_k{n_cluster}_means_raw.h5', overwrite=True)
        np.save(f'{save_dir}/{feat_name}_k{n_cluster}_means_getoutput.npy', cluster_getoutput)
        np.save(f'{save_dir}/{feat_name}_k{n_cluster}_means_dtrajs.npy', cluster_dtrajs)
        np.save(f'{save_dir}/{feat_name}_k{n_cluster}_means_centers.npy', cluster_centers)
    return cluster, cluster_centers

def generate_msm(cluster, n_cluster, msm_lag, save, save_dir, save_name, feat_name):
    msm = pyemma.msm.estimate_markov_model(cluster.dtrajs, msm_lag)
    if save == True:
        state_frac_used = msm.active_state_fraction
        count_frac_used = msm.active_count_fraction
        feat_act_set = msm.active_set
        eigvec = msm.eigenvectors_right()

        log_save = open(f'{save_dir}/{feat_name}_k{n_cluster}_msm_params.log','w')
        log_save.write(f'Fraction of states used: {state_frac_used}.\nFraction of counts used: {count_frac_used}. \nMain indices of the active states (members of the largest connected set): {list(feat_act_set)}. \nStationary distribution across states: {msm.stationary_distribution}. \nFirst eigenvector is one: {np.allclose(eigvec[:, 0], 1, atol=1e-15)} (min={eigvec[:, 0].min()}, max={eigvec[:, 0].max()})')
        log_save.close()
        msm.save(f'{save_dir}/{feat_name}_k{n_cluster}_msm_raw.h5',  overwrite=True)
    return msm
